# Agent: Feedback

이 튜토리얼을 통해 다음을 배울 수 있다:

- 인간 참여형(Human-in-the-loop) 워크플로우의 필요성을 이해한다.
- 승인 및 거부 메커니즘을 구현한다.
- 피드백 기반 개선 루프를 만든다.
- 다양한 승인 패턴을 실전에 적용한다.

## 1. 환경 설정

### 1.1 필요한 라이브러리 설치

In [ ]:
%pip install openai python-dotenv

### 1.2 환경 설정

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### 1.3 필요한 라이브러리 가져오기

In [4]:
from openai import OpenAI
from typing import Optional
from enum import Enum

MODEL = "gpt-4o-mini"

client = OpenAI()

## 2. 기본 승인 워크플로우

### 2.1 단순 승인 시스템
가장 기본적인 승인/거부 메커니즘이다.

In [5]:
def get_human_approval(content: str) -> bool:
    """사용자로부터 승인 여부를 입력받는 함수다."""
    print(f"\n생성된 내용:\n{content}\n")
    print("-" * 50)
    
    while True:
        response = input("승인하시겠습니까? (y/n): ").lower().strip()
        if response in ['y', 'yes']:
            return True
        elif response in ['n', 'no']:
            return False
        else:
            print("잘못된 입력입니다. 'y' 또는 'n'으로 답해주세요.")

def generate_with_approval(prompt: str) -> Optional[str]:
    """승인이 포함된 생성 함수다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    
    draft = response.choices[0].message.content
    approved = get_human_approval(draft)
    
    status = "승인됨" if approved else " 거부됨"
    print(f"\n{status}")
    
    return draft if approved else None

# 사용 예시
result = generate_with_approval("기술에 대한 짧은 시를 작성해줘.")
if result:
    print("\n--- 최종 결과물 ---\n", result)


생성된 내용:
기술의 바다, 광활한 꿈,  
코드의 파도, 무한한 회상.  
손끝으로 여는 새로운 세계,  
정보의 빛, 어둠을 가르며.  

기계의 노래, 진화를 담고,  
인간의 마음, 사이를 잇고.  
서로의 손을, 디지털로 감고,  
미래를 향해, 함께 나가리.  

--------------------------------------------------


승인하시겠습니까? (y/n):  y



승인됨

--- 최종 결과물 ---
 기술의 바다, 광활한 꿈,  
코드의 파도, 무한한 회상.  
손끝으로 여는 새로운 세계,  
정보의 빛, 어둠을 가르며.  

기계의 노래, 진화를 담고,  
인간의 마음, 사이를 잇고.  
서로의 손을, 디지털로 감고,  
미래를 향해, 함께 나가리.  


#### 워크플로우:
1. LLM이 초안을 생성한다.
2. 사용자에게 내용을 보여준다.
3. 승인/거부를 입력받는다.
4. 승인된 경우에만 결과를 반환한다.

### 2.2 개선 루프가 있는 승인
거부 시 피드백을 받아 개선하고 재시도하는 시스템이다.

In [6]:
def generate_with_improvement_loop(prompt: str, max_iterations: int = 3) -> Optional[str]:
    """개선 루프가 포함된 생성 함수다."""
    
    messages = [{"role": "user", "content": prompt}]
    
    for iteration in range(max_iterations):
        print(f"\n=== 시도 {iteration + 1}/{max_iterations} ===")
        
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        
        draft = response.choices[0].message.content
        messages.append({"role": "assistant", "content": draft})
        
        approved = get_human_approval(draft)
        
        if approved:
            print("최종 승인")
            return draft
        
        if iteration < max_iterations - 1:
            feedback = input("개선 사항을 입력하세요: ")
            messages.append({"role": "user", "content": f"피드백: {feedback}. 이 피드백을 반영하여 수정해주세요."})
    
    print(" 최대 시도 횟수에 도달했습니다.")
    return None

# 사용 예시
result = generate_with_improvement_loop("우리 AI 회사를 소개하는 짧은 이메일 초안을 작성해줘")


=== 시도 1/3 ===

생성된 내용:
물론입니다! 아래는 AI 회사를 소개하는 짧은 이메일 초안입니다.

---

제목: [회사명] 소개 및 협력 제안

안녕하세요 [수신자 이름]님,

저는 [회사명]의 [직책] [이름]입니다. 저희 회사는 최신 AI 기술을 활용하여 [특정 분야/산업]에서 혁신적인 솔루션을 제공하는 전문 기업입니다. 우리는 [주요 제품/서비스]를 통해 고객의 문제를 해결하고, 업무 효율성을 향상시키는 데 기여하고 있습니다.

[회사명]의 솔루션은 [특정 장점 또는 차별점]를 가지고 있어, 고객에게 실질적인 가치를 제공합니다. 앞으로 [수신자 회사명]와의 협력을 통해 더욱 큰 시너지를 낼 수 있기를 기대합니다.

혹시 더 궁금한 점이나 논의하고 싶은 사항이 있으시면 언제든지 연락 주시기 바랍니다. 귀하의 소중한 의견을 기다리고 있겠습니다.

감사합니다.

[이름]  
[직책]  
[회사명]  
[연락처]  
[웹사이트]

--- 

이메일 내용을 필요에 따라 수정하시면 됩니다. 도움이 필요하시면 말씀해 주세요!

--------------------------------------------------


승인하시겠습니까? (y/n):  y


최종 승인


### 2.3 상세 피드백 수집
단순 승인/거부를 넘어 상세한 피드백을 받는다.

In [7]:
class FeedbackType(str, Enum):
    """피드백 유형이다."""
    APPROVE = "approve"
    REVISE = "revise"
    REJECT = "reject"

class DetailedFeedback:
    """상세 피드백을 수집하는 클래스다."""
    
    def __init__(self):
        self.content = None
        self.feedback_type = None
        self.comments = None
        self.rating = None
    
    def collect(self, content: str):
        """피드백을 수집한다."""
        self.content = content
        print(f"\n생성된 내용:\n{content}\n")
        print("-" * 50)
        
        print("\n옵션을 선택하세요:")
        print("1. 승인 (approve)")
        print("2. 수정 요청 (revise)")
        print("3. 거부 (reject)")
        
        choice = input("선택 (1/2/3): ")
        
        feedback_map = {"1": FeedbackType.APPROVE, "2": FeedbackType.REVISE, "3": FeedbackType.REJECT}
        self.feedback_type = feedback_map.get(choice, FeedbackType.REVISE)
        
        if self.feedback_type != FeedbackType.APPROVE:
            self.comments = input("피드백 내용: ")
        
        while True:
            try:
                rating_input = input("평점 (1-5): ")
                self.rating = int(rating_input)
                if 1 <= self.rating <= 5:
                    break
                else:
                    print("1에서 5 사이의 숫자를 입력해주세요.")
            except ValueError:
                print("숫자를 입력해주세요.")
        
        return self.feedback_type == FeedbackType.APPROVE

# 사용 예시
feedback_collector = DetailedFeedback()
draft = "여기에 AI가 생성한 마케팅 문구가 들어갑니다."
approved = feedback_collector.collect(draft)
print(f"\n수집된 피드백: {feedback_collector.feedback_type.value}")
print(f"코멘트: {feedback_collector.comments}")
print(f"평점: {feedback_collector.rating}")


생성된 내용:
여기에 AI가 생성한 마케팅 문구가 들어갑니다.

--------------------------------------------------

옵션을 선택하세요:
1. 승인 (approve)
2. 수정 요청 (revise)
3. 거부 (reject)


선택 (1/2/3):  1
평점 (1-5):  5



수집된 피드백: approve
코멘트: None
평점: 5


## 3. 고급 피드백 패턴

### 3.1 다단계 승인 프로세스
여러 사람의 승인이 필요한 경우를 처리한다.

In [8]:
class MultiStageApproval:
    """다단계 승인 시스템이다."""
    
    def __init__(self, approvers: list[str]):
        self.approvers = approvers
        self.approvals = {}
    
    def request_approval(self, content: str, stage: str) -> bool:
        """특정 단계의 승인을 요청한다."""
        print(f"\n=== {stage} 승인 단계 ===")
        print(f"내용:\n{content}\n")
        
        response = input(f"{stage} 승인 (y/n): ")
        approved = response.lower().startswith("y")
        
        self.approvals[stage] = approved
        return approved
    
    def process(self, content: str) -> bool:
        """모든 단계의 승인을 처리한다."""
        for approver in self.approvers:
            approved = self.request_approval(content, approver)
            
            if not approved:
                print(f" {approver}에게 거부됨")
                return False
        
        print(" 모든 승인 완료")
        return True

# 사용 예시
approval_system = MultiStageApproval(["팀장", "부서장", "법무팀"])
content = "새로운 서비스 이용 약관 초안입니다."
final_approved = approval_system.process(content)


=== 팀장 승인 단계 ===
내용:
새로운 서비스 이용 약관 초안입니다.



팀장 승인 (y/n):  y



=== 부서장 승인 단계 ===
내용:
새로운 서비스 이용 약관 초안입니다.



부서장 승인 (y/n):  y



=== 법무팀 승인 단계 ===
내용:
새로운 서비스 이용 약관 초안입니다.



법무팀 승인 (y/n):  y


 모든 승인 완료


### 3.2 조건부 승인
특정 조건에 따라 자동/수동 승인을 결정한다.

In [9]:
class ConditionalApproval:
    """조건부 승인 시스템이다."""
    
    def __init__(self, auto_approve_threshold: float = 0.9):
        self.threshold = auto_approve_threshold
        self.client = OpenAI()
    
    def assess_confidence(self, content: str) -> float:
        """콘텐츠 생성의 자신감 점수를 평가한다."""
        prompt = f"""
        다음 콘텐츠가 얼마나 사실에 기반하고 비즈니스적으로 안전한지 자신감 점수를 0.0에서 1.0 사이로 평가하라.
        0.0 = 매우 불확실하고 위험, 1.0 = 매우 확실하고 안전
        
        콘텐츠: {content}
        
        숫자만 반환하라.
        """
        
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": prompt}]
        )
        
        try:
            confidence_score = float(response.choices[0].message.content.strip())
            return confidence_score
        except ValueError:
            return 0.0 # 파싱 실패 시 가장 안전한 점수로 처리한다
    
    def process(self, content: str) -> bool:
        """자신감 점수에 따라 승인을 처리한다."""
        confidence = self.assess_confidence(content)
        print(f"자신감 점수 평가: {confidence:.2f}")
        
        if confidence >= self.threshold:
            print(f"자동 승인됨 (자신감 점수 {self.threshold} 이상)")
            return True
        else:
            print(f"수동 승인 필요 (자신감 점수 {self.threshold} 미만)")
            return get_human_approval(content)

# 사용 예시
conditional_approval = ConditionalApproval(auto_approve_threshold=0.9)
approved = conditional_approval.process("오늘 서울의 날씨는 맑습니다. 기온은 25도입니다.")
print("최종 승인 상태:", approved)

자신감 점수 평가: 0.70
수동 승인 필요 (자신감 점수 0.9 미만)

생성된 내용:
오늘 서울의 날씨는 맑습니다. 기온은 25도입니다.

--------------------------------------------------


승인하시겠습니까? (y/n):  y


최종 승인 상태: True


### 3.3 피드백 기반 학습
피드백을 수집하여 향후 생성을 개선한다.

In [10]:
class FeedbackLearningSystem:
    """피드백 기반 학습 시스템이다."""
    
    def __init__(self):
        self.feedback_history = []
        self.client = OpenAI()
    
    def generate_with_history(self, prompt: str) -> str:
        """과거 피드백을 반영하여 생성한다."""
        context = self._build_context()
        
        full_prompt = f"""
        사용자 요청: {prompt}
        
        --- 과거 피드백 ---
        {context}
        ---------------------
        
        과거에 거부되었던 피드백을 참고하여 더 나은 결과를 생성하라.
        """
        
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": full_prompt}]
        )
        
        return response.choices[0].message.content
    
    def collect_feedback(self, content: str, approved: bool, comments: str = ""):
        """피드백을 수집하고 저장한다."""
        self.feedback_history.append({
            "content": content,
            "approved": approved,
            "comments": comments
        })
        print("피드백이 기록되었습니다.")
    
    def _build_context(self) -> str:
        """피드백 히스토리에서 컨텍스트를 구축한다."""
        if not self.feedback_history:
            return "없음"
        
        recent_feedback = self.feedback_history[-3:]
        context_parts = []
        
        for fb in recent_feedback:
            if not fb['approved']:
                context_parts.append(f"- 거부된 내용: '{fb['content'][:30]}...' -> 피드백: {fb['comments']}")
        
        return "\n".join(context_parts) if context_parts else "최근 거부된 피드백 없음"

# 사용 예시
learning_system = FeedbackLearningSystem()

for i in range(3):
    print(f"\n=== 라운드 {i + 1} ===")
    draft = learning_system.generate_with_history("우리 제품의 장점을 강조하는 광고 문구를 작성해줘")
    approved = get_human_approval(draft)
    comments = ""
    if not approved:
        comments = input("피드백 코멘트: ")
    learning_system.collect_feedback(draft, approved, comments)


=== 라운드 1 ===

생성된 내용:
물론입니다! 제품의 장점을 강조하는 광고 문구를 몇 가지 제안드립니다.

1. "우리 제품, 당신의 일상을 변화시킵니다! 효율성과 품질을 동시에 잡은 스마트한 선택!"
  
2. "끝없는 가능성, 우리의 제품으로 시작하세요! 혁신적인 성능과 세련된 디자인이 함께합니다."

3. "고객의 만족을 최우선으로! 신뢰할 수 있는 품질을 가진 우리의 제품으로 더 나은 내일을 만나세요."

4. "특별한 순간을 위한 최상의 선택! 우리의 제품으로 더욱 빛나는 일상을 경험하세요."

5. "신뢰할 수 있는 기술과 탁월한 성능! 우리의 제품이 당신의 기대를 뛰어넘습니다."

필요에 따라 조정하거나 조합하여 사용하시면 좋습니다!

--------------------------------------------------


승인하시겠습니까? (y/n):  y


피드백이 기록되었습니다.

=== 라운드 2 ===

생성된 내용:
물론입니다! 제품의 장점을 강조하는 광고 문구를 다음과 같이 제안드립니다:

"당신의 삶을 한층 더 풍요롭게! 우리의 제품은 탁월한 품질과 혁신적인 디자인으로, 매일의 소소한 순간들을 특별하게 만들어드립니다. 지금 바로 경험해보세요!" 

어떤 특정 장점이나 특징을 더 강조하길 원하신다면 말씀해 주세요!

--------------------------------------------------


승인하시겠습니까? (y/n):  y


피드백이 기록되었습니다.

=== 라운드 3 ===

생성된 내용:
물론입니다! 제품의 장점을 강조하는 광고 문구를 아래와 같이 제안드립니다:

1. "당신의 생활을 더욱 빛나게! 우리 제품으로 매일매일 특별한 경험을 누려보세요."
   
2. "탁월한 품질, 눈에 띄는 효과! 우리 제품으로 진정한 변화를 경험하세요."

3. "더 나은 내일을 위한 스마트한 선택! 우리 제품이 당신의 삶을 한층 더 풍요롭게 만들어 드립니다."

4. "신뢰할 수 있는 혁신! 우리가 만든 제품이 당신의 기대를 넘어서겠습니다."

5. "가장 소중한 순간을 위해, 저희 제품이 함께합니다. 당신의 행복을 위한 최상의 선택!"

이런 문구들이 제품의 장점을 효과적으로 강조하는 데 도움이 되길 바랍니다. 필요에 따라 조정하거나 추가 요청해 주세요!

--------------------------------------------------


승인하시겠습니까? (y/n):  n
피드백 코멘트:  없음


피드백이 기록되었습니다.


## 4. 실전 예제

### 4.1 이메일 발송 승인 시스템
이메일을 보내기 전에 검토하고 승인한다.

In [11]:
class EmailApprovalSystem:
    """이메일 발송 승인 시스템이다."""
    
    def __init__(self):
        self.client = OpenAI()
    
    def draft_email(self, recipient: str, purpose: str) -> dict:
        """이메일 초안을 작성한다."""
        prompt = f"""
        수신자: {recipient}
        목적: {purpose}
        
        위 정보를 바탕으로 전문적이고 친절한 이메일의 제목과 본문을 작성하라. 제목은 '제목: ' 다음에, 본문은 '본문: ' 다음에 작성하라.
        """
        
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": prompt}]
        )
        
        content = response.choices[0].message.content
        # 제목과 본문을 분리한다
        subject = "자동 생성된 제목"
        body = content
        try:
            subject = content.split("제목: ")[1].split("\n")[0]
            body = content.split("본문: ")[1]
        except IndexError:
            print("제목/본문 파싱에 실패하여 전체 내용을 본문으로 사용합니다.")
        
        email = {
            "recipient": recipient,
            "subject": subject,
            "body": body,
            "status": "pending"
        }
        
        return email
    
    def review_and_send(self, email: dict) -> bool:
        """이메일을 검토하고 발송 여부를 결정한다."""
        full_content = f"수신자: {email['recipient']}\n제목: {email['subject']}\n\n{email['body']}"
        
        approved = get_human_approval(full_content)
        
        if approved:
            email["status"] = "sent"
            print("\n 이메일이 발송되었습니다.")
        else:
            email["status"] = "rejected"
            print("\n 이메일 발송이 취소되었습니다.")
        
        return approved

# 사용 예시
email_system = EmailApprovalSystem()
draft = email_system.draft_email("customer@example.com", "신제품 출시 안내")
email_system.review_and_send(draft)


생성된 내용:
수신자: customer@example.com
제목: 신제품 출시 안내


안녕하세요, 고객님.

저희 [회사명]에서 새롭게 출시된 제품에 대해 안내드리게 되어 매우 기쁩니다. 고객님의 기대에 부응하기 위해 오랜 시간 동안 면밀히 연구하고 개발한 [신제품명]은 혁신적인 기술과 세련된 디자인으로 만들어졌습니다. 

[신제품명]은 다음과 같은 특징을 가지고 있습니다:
- [특징 1]
- [특징 2]
- [특징 3]

출시일은 [출시일]이며, 지금 주문하시는 고객님께는 특별 할인 혜택이 제공됩니다. 자세한 정보와 구매는 저희 웹사이트([웹사이트 링크])에서 확인하실 수 있습니다.

고객님의 소중한 의견과 관심을 항상 기다리고 있습니다. 추가적인 질문이나 요청사항이 있으시면 언제든지 연락 주시기 바랍니다.

감사합니다.

[회사명] 드림  
[연락처]  
[웹사이트 링크]

--------------------------------------------------


승인하시겠습니까? (y/n):  ㅛ


잘못된 입력입니다. 'y' 또는 'n'으로 답해주세요.


승인하시겠습니까? (y/n):  y



 이메일이 발송되었습니다.


True

## 5. 피드백 모범 사례

### 5.1 명확한 컨텍스트 제공
승인자가 충분한 정보를 바탕으로 결정할 수 있게 한다.

In [12]:
class ContextualApproval:
    """컨텍스트가 풍부한 승인 시스템이다."""
    
    def show_with_context(self, content: str, metadata: dict) -> bool:
        """컨텍스트와 함께 내용을 표시한다."""
        print("\n" + "=" * 50)
        print("컨텍스트 정보:")
        for key, value in metadata.items():
            print(f"  - {key}: {value}")
        print("=" * 50)
        
        return get_human_approval(content)

# 사용 예시
contextual_approval = ContextualApproval()
approved = contextual_approval.show_with_context(
    "[AI 생성 콘텐츠] 다음 분기 마케팅 캠페인은 '혁신'을 주제로 진행합니다.",
    {
        "생성 일시": "2025-10-26 14:30",
        "요청자": "마케팅팀",
        "대상 고객": "기업 임원",
        "위험도 평가": "중간"
    }
)


컨텍스트 정보:
  - 생성 일시: 2025-10-26 14:30
  - 요청자: 마케팅팀
  - 대상 고객: 기업 임원
  - 위험도 평가: 중간

생성된 내용:
[AI 생성 콘텐츠] 다음 분기 마케팅 캠페인은 '혁신'을 주제로 진행합니다.

--------------------------------------------------


승인하시겠습니까? (y/n):  y


### 5.2 승인 기록 유지
모든 승인 결정을 추적 가능하게 기록한다.

In [13]:
from datetime import datetime

class ApprovalLogger:
    """승인 기록을 유지하는 클래스다."""
    
    def __init__(self):
        self.logs = []
    
    def log_approval(self, content: str, approved: bool, approver: str, comments: str = ""):
        """승인 결정을 기록한다."""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "content_preview": content[:50] + "...",
            "approved": approved,
            "approver": approver,
            "comments": comments
        }
        self.logs.append(log_entry)
    
    def print_summary(self):
        """승인 기록 요약을 출력한다."""
        total = len(self.logs)
        if total == 0:
            print("기록이 없습니다.")
            return
            
        approved_count = sum(1 for log in self.logs if log["approved"])
        
        print(f"\n=== 승인 기록 요약 ===")
        print(f"총 요청: {total}")
        print(f"승인: {approved_count}")
        print(f"거부: {total - approved_count}")
        print(f"승인율: {approved_count/total*100:.1f}%")

# 사용 예시
logger = ApprovalLogger()
logger.log_approval("콘텐츠 1: AI의 미래에 대한 블로그 글", True, "김팀장", "좋음")
logger.log_approval("콘텐츠 2: 신제품 출시 이메일 초안", False, "이과장", "문구가 너무 공격적임")
logger.log_approval("콘텐츠 3: 주간 보고서 요약", True, "박대리", "")
logger.print_summary()


=== 승인 기록 요약 ===
총 요청: 3
승인: 2
거부: 1
승인율: 66.7%


### 5.3 피드백 템플릿
일관된 피드백을 수집하기 위한 템플릿을 사용한다.

In [14]:
class FeedbackTemplate:
    """구조화된 피드백 템플릿이다."""
    
    def __init__(self):
        self.categories = {
            "정확성": "내용이 사실에 기반하고 정확한가?",
            "적절성": "대상 청중과 목적에 적절한가?",
            "톤앤매너": "브랜드 가이드라인에 맞는 톤앤매너인가?",
            "완성도": "문법적 오류 없이 완성도가 높은가?"
        }
    
    def collect_structured_feedback(self, content: str) -> dict:
        """구조화된 피드백을 수집한다."""
        print(f"\n내용:\n{content}\n")
        print("=" * 50)
        
        feedback = {}
        
        for category, question in self.categories.items():
            print(f"\n{category}: {question}")
            while True:
                try:
                    rating = int(input("평가 (1-5): "))
                    if 1 <= rating <= 5:
                        feedback[category] = rating
                        break
                    else:
                        print("1에서 5 사이의 숫자를 입력해주세요.")
                except ValueError:
                    print("숫자로 입력해주세요.")
        
        overall = input("\n전체 승인 여부 (y/n): ")
        feedback["approved"] = overall.lower().startswith("y")
        
        return feedback

# 사용 예시
template = FeedbackTemplate()
feedback = template.collect_structured_feedback("새로운 마케팅 캠페인 슬로건: '미래를 코딩하라, 지금 바로!'")
print(f"\n수집된 피드백: {feedback}")


내용:
새로운 마케팅 캠페인 슬로건: '미래를 코딩하라, 지금 바로!'


정확성: 내용이 사실에 기반하고 정확한가?


평가 (1-5):  5



적절성: 대상 청중과 목적에 적절한가?


평가 (1-5):  4



톤앤매너: 브랜드 가이드라인에 맞는 톤앤매너인가?


평가 (1-5):  5



완성도: 문법적 오류 없이 완성도가 높은가?


평가 (1-5):  4

전체 승인 여부 (y/n):  3



수집된 피드백: {'정확성': 5, '적절성': 4, '톤앤매너': 5, '완성도': 4, 'approved': False}
